**Data Pre-processing**

*The process should be: Preprocess each table → Merge → Post-merge refinement*

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime


# 1. set the path
DATA_PATH = r'c:\Users\Jack\Desktop\GroupWork\Code&Data\Data\raw'
OUTPUT_PATH = r'c:\Users\Jack\Desktop\GroupWork\Code&Data\Data\processed'

print("=" * 60)
print("CRISP-DM Data Pre-processing")
print("=" * 60)

# ensure the output directory exists
os.makedirs(OUTPUT_PATH, exist_ok=True)

# 2. load the data
print("\n[Step 1] Load the original data...")

datasets = {}
files = [
    ('assessments', 'assessments.csv'),
    ('courses', 'courses.csv'),
    ('studentAssessment', 'studentAssessment.csv'),
    ('studentInfo', 'studentInfo.csv'),
    ('studentRegistration', 'studentRegistration.csv'),
    ('studentVle', 'studentVle.csv'),
    ('vle', 'vle.csv')
]

for key, filename in files:
    filepath = os.path.join(DATA_PATH, filename)
    datasets[key] = pd.read_csv(filepath)
    print(f"loaded {key}: {datasets[key].shape[0]:,} rows x {datasets[key].shape[1]} columns")

CRISP-DM Data Pre-processing

[Step 1] Load the original data...
loaded assessments: 206 rows x 6 columns
loaded courses: 22 rows x 3 columns
loaded studentAssessment: 173,912 rows x 5 columns
loaded studentInfo: 32,593 rows x 12 columns
loaded studentRegistration: 32,593 rows x 5 columns
loaded studentVle: 10,655,280 rows x 6 columns
loaded vle: 6,364 rows x 6 columns


*Clean First*

In [2]:
# 3. 独立表清洗 (Clean First)
print("\n[Step 2] Clean the data of independent tables...")

# ---------- 3.1 assessments 清洗 ----------
print("\n  [2.1] 清洗 assessments...")
assessments = datasets['assessments'].copy()

# 验证：缺失的date是否全部属于"Exam"类型
missing_date_exams = assessments[assessments['date'] == -1]['assessment_type'].value_counts()

# 如果存在非Exam类型的缺失date，发出警告
if 'Exam' in missing_date_exams.index and missing_date_exams.sum() > missing_date_exams.get('Exam', 0):
    print("存在非Exam类型的date缺失！")
else:
    print("所有缺失date的记录均为Exam类型")

# 处理缺失date (11条, 5.34%)
assessments['date'] = assessments['date'].fillna(-1)  # -1表示无日期信息
assessments['date'] = assessments['date'].astype(int)
if assessments['date'].isnull().sum() == 0:
    print("There is no any dulipcated data")

# 验证权重总和 (每门课程应该约等于100)
assessments['weight'] = pd.to_numeric(assessments['weight'], errors='coerce')

# 1. 打印范围
print(f"    - 权重范围: {assessments['weight'].min():.2f} - {assessments['weight'].max():.2f}")

# 2. 详细拆分：Exam vs Others (TMA + CMA)
print("    - 权重明细 (Exam vs Others):")

# 提取 Exam 权重 (如果有多场考试，用列表或逗号分隔显示)
exam_weights = assessments[assessments['assessment_type'] == 'Exam'].groupby(
    ['code_module', 'code_presentation']
)['weight'].apply(lambda x: ", ".join(x.astype(str).unique()))

# 计算 Others 权重总和
other_weights = assessments[assessments['assessment_type'] != 'Exam'].groupby(
    ['code_module', 'code_presentation']
)['weight'].sum()

# 合并展示
for (module, pres), other in other_weights.items():
    exam = exam_weights.get((module, pres), "N/A")
    print(f"      {module}/{pres}: Exam={exam}, Others={other:.2f}")


[Step 2] Clean the data of independent tables...

  [2.1] 清洗 assessments...
所有缺失date的记录均为Exam类型
There is no any dulipcated data
    - 权重范围: 0.00 - 100.00
    - 权重明细 (Exam vs Others):
      AAA/2013J: Exam=100.0, Others=100.00
      AAA/2014J: Exam=100.0, Others=100.00
      BBB/2013B: Exam=100.0, Others=100.00
      BBB/2013J: Exam=100.0, Others=100.00
      BBB/2014B: Exam=100.0, Others=100.00
      BBB/2014J: Exam=100.0, Others=100.00
      CCC/2014B: Exam=100.0, Others=100.00
      CCC/2014J: Exam=100.0, Others=100.00
      DDD/2013B: Exam=100.0, Others=100.00
      DDD/2013J: Exam=100.0, Others=100.00
      DDD/2014B: Exam=100.0, Others=100.00
      DDD/2014J: Exam=100.0, Others=100.00
      EEE/2013J: Exam=100.0, Others=100.00
      EEE/2014B: Exam=100.0, Others=100.00
      EEE/2014J: Exam=100.0, Others=100.00
      FFF/2013B: Exam=100.0, Others=100.00
      FFF/2013J: Exam=100.0, Others=100.00
      FFF/2014B: Exam=100.0, Others=100.00
      FFF/2014J: Exam=100.0, Others=100.00

- Considering that the null values are from the exam, the explanation in the question states: If the information about the final exam date is missing, it is at the end of the last presentation week.
- Therefore, after considering the merging, the Date processing should be carried out. First, set it to -1.
- For weight, only the multiples of 100 are needed to be considered. One exam is 100%, and the sum of other evaluations is 100%.

### The Course table is free of missing and duplicate, and relationships are processed after merging

Consider the b/J relation given in the problem

In [3]:
studentAssessment = datasets['studentAssessment'].copy()
courses = datasets['courses'].copy()
studentInfo = datasets['studentInfo'].copy()

The remaining Categorical data will be processed by One-Hot encoding after merging

In [4]:
# ---------- 3.4 studentRegistration 清洗 ----------
print("\n  [2.4] 清洗 studentRegistration...")
studentRegistration = datasets['studentRegistration'].copy()

# date_registration缺失 (45条, 0.14%) - 使用同一门课程/课程序列的平均值填补
# 计算每个 (code_module, code_presentation) 的平均注册时间
mean_reg_by_course = studentRegistration.groupby(['code_module', 'code_presentation'])['date_registration'].transform('mean')

# 用均值填补缺失值，保留已有值
studentRegistration['date_registration'] = studentRegistration['date_registration'].fillna(mean_reg_by_course)

# 展示填补情况
missing_count = datasets['studentRegistration']['date_registration'].isna().sum()
print(f"    - 缺失date_registration使用课程均值填补 (共{missing_count}条)")


  [2.4] 清洗 studentRegistration...
    - 缺失date_registration使用课程均值填补 (共45条)


When the student successfully completes the course, unregi is empty, but if you are not completely sure that he dropped out of the course, it is best to decide by final_result, and wait until after synthesizing the table

In [5]:
# ---------- 3.5 studentAssessment 清洗 ---------- (clean it after merging the data)
# print("\n  [2.5] 清洗 studentAssessment...")
# studentAssessment = datasets['studentAssessment'].copy()

# # 1. 缺失分数统计
# missing_score_count = studentAssessment['score'].isnull().sum()
# print(f"    - 发现 {missing_score_count} 个缺失分数")

# # 涉及多少 assessment
# missing_assessments = studentAssessment.loc[
#     studentAssessment['score'].isnull(), 'id_assessment'
# ].unique()
# print(f"    - 涉及 {len(missing_assessments)} 个评估项目")

# # 2. 按每个 assessment 的均值填补缺失分数
# studentAssessment['score_filled'] = (
#     studentAssessment
#     .groupby('id_assessment')['score']
#     .transform(lambda x: x.fillna(x.mean()))
# )

# print("    - 已使用每个 assessment 的均值填补缺失分数")

# # 3. 检测异常分数（仅报告，不修改原始值）
# abnormal_scores = studentAssessment[
#     (studentAssessment['score'] < 0) | (studentAssessment['score'] > 100)
# ]

# print(f"    - 发现 {len(abnormal_scores)} 条异常分数记录")

# if len(abnormal_scores) > 0:
#     print("    - 异常样本示例：")
#     print(abnormal_scores[['id_student', 'id_assessment', 'score']].head())

The future needs to see if the test scores and whether students drop out have any effect

In [6]:
# ---------- 3.6 studentVle 清洗 (最复杂) ----------
from pandas.core.frame import duplicated


print("\n  [2.6] 清洗 studentVle...")
studentVle = datasets['studentVle'].copy()

# 原始数据量
original_rows = len(studentVle)
print(f"    - 原始数据量: {original_rows:,} 行")

student_site_level = (
    studentVle.groupby(["id_student", "id_site", "code_module","code_presentation"])
      .agg(
          total_clicks=("sum_click", "sum"),
          first_day=("date", "min"),
          last_day=("date", "max")
      )
      .reset_index()
)

dedup_rows = len(student_site_level)
dedup_count = original_rows - dedup_rows
print(f"    - 去重后数据量: {dedup_rows:,} 行 (减少 {dedup_count:,} 条)")

# release the menory
studentVle = None
studentVle_clean = student_site_level
print(f"    - 已完成重复记录合并")

# 最终检测重复行数
dup_check = studentVle_clean.duplicated().sum()
print(f"    - 最终重复行检测: {dup_check} 条")
studentVle_clean.head(10)


  [2.6] 清洗 studentVle...
    - 原始数据量: 10,655,280 行
    - 去重后数据量: 1,960,496 行 (减少 8,694,784 条)
    - 已完成重复记录合并
    - 最终重复行检测: 0 条


,id_student,id_site,code_module,code_presentation,total_clicks,first_day,last_day
0,6516,877011,AAA,2014J,23,204,218
1,6516,877012,AAA,2014J,305,0,231
2,6516,877015,AAA,2014J,26,50,231
3,6516,877023,AAA,2014J,6,0,184
4,6516,877025,AAA,2014J,91,-22,227
5,6516,877030,AAA,2014J,497,-23,269
6,6516,877031,AAA,2014J,16,75,80
7,6516,877032,AAA,2014J,11,223,223
8,6516,877033,AAA,2014J,7,74,141
9,6516,877034,AAA,2014J,7,28,62


In [7]:
vle = datasets['vle'].copy()
print("\n  [2.7] cleaning VLE...")
vle = datasets["vle"].copy()

# 原始数据量
original_rows = len(vle)
print(f"    - original_rows: {original_rows:,} 行")

# 只删除 week_from 和 week_to
vle_clean = vle.drop(columns=["week_from", "week_to"], errors="ignore")

print("delete sucussful")

# 简单一致性检查
dup_rows = vle_clean.duplicated().sum()
print(f"    - 当前重复行数: {dup_rows:,} 行")



  [2.7] cleaning VLE...
    - original_rows: 6,364 行
delete sucussful
    - 当前重复行数: 0 行


In [8]:
print("\n[merge] merge activity_type into studentVle_clean ...")


[merge] merge activity_type into studentVle_clean ...


In [9]:
# merge
studentVle_clean = studentVle_clean.merge(
    vle_clean,
    on=["id_site", "code_module", "code_presentation"],
    how="left"
)

# 合并结果检查
miss = studentVle_clean["activity_type"].isna().sum()
print(f"Finish")
print(miss)
studentVle_clean.head(10)

Finish
0


,id_student,id_site,code_module,code_presentation,total_clicks,first_day,last_day,activity_type
0,6516,877011,AAA,2014J,23,204,218,forumng
1,6516,877012,AAA,2014J,305,0,231,forumng
2,6516,877015,AAA,2014J,26,50,231,forumng
3,6516,877023,AAA,2014J,6,0,184,forumng
4,6516,877025,AAA,2014J,91,-22,227,forumng
5,6516,877030,AAA,2014J,497,-23,269,homepage
6,6516,877031,AAA,2014J,16,75,80,oucontent
7,6516,877032,AAA,2014J,11,223,223,oucontent
8,6516,877033,AAA,2014J,7,74,141,oucontent
9,6516,877034,AAA,2014J,7,28,62,oucontent


In [10]:
# # 1. 先做 pivot：activity_type → 各类点击次数
# click_features = studentVle_clean.pivot_table(
#     index=["id_student", "code_module", "code_presentation"],
#     columns="activity_type",
#     values="total_clicks",
#     aggfunc="sum",
#     fill_value=0
# )

# # 重命名列，加上 _clicks 后缀
# click_features.columns = [f"{c}_clicks" for c in click_features.columns]

# # 2. 计算每个学生在该课程中的总体 first_day 和 last_day
# time_features = (
#     studentVle_clean
#     .groupby(["id_student", "code_module", "code_presentation"])
#     .agg(
#         first_day=("first_day", "min"),
#         last_day=("last_day", "max")
#     )
# )

# # 3. 合并成最终特征表
# studentVle_final = (
#     click_features
#     .merge(time_features, left_index=True, right_index=True)
#     .reset_index()
# )
# studentVle_final.head(10)

In [11]:
studentVle_final = (
    studentVle_clean
    .groupby(["id_student", "code_module", "code_presentation"], as_index=False)
    .agg(
        total_clicks=("total_clicks", "sum"),
        first_day=("first_day", "min"),
        last_day=("last_day", "max")
    )
)

studentVle_final.head(10)

,id_student,code_module,code_presentation,total_clicks,first_day,last_day
0,6516,AAA,2014J,2791,-23,269
1,8462,DDD,2013J,646,-6,118
2,8462,DDD,2014J,10,10,10
3,11391,AAA,2013J,934,-5,253
4,23629,BBB,2013B,161,-6,87
5,23698,CCC,2014J,910,-18,231
6,23798,BBB,2013J,590,-4,267
7,24186,GGG,2014B,184,-4,240
8,24213,DDD,2014B,1992,-11,237
9,24391,GGG,2013J,712,2,249


In [12]:
studentVle_final.shape

(29228, 6)

In [13]:
merged_assessment = studentAssessment.merge(
    assessments,
    on="id_assessment",
    how="left"
)

print("Data shape:", merged_assessment.shape)
merged_assessment.head()

Data shape: (173912, 10)


,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19,10.0
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19,10.0
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19,10.0
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19,10.0
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19,10.0


In [14]:
df = merged_assessment.drop(columns=["assessment_type", "date"], errors="ignore").copy()

course_weighted_sum = (
    df
    .assign(weighted_component=df["score"] * df["weight"] / 100.0)
    .groupby(["id_student", "code_module", "code_presentation"])
    .agg(
        weighted_score=("weighted_component", "sum"),
        total_weight_attempted=("weight", "sum"),
        n_assessments=("id_assessment", "nunique")
    )
    .reset_index()
)

print(course_weighted_sum.shape)
course_weighted_sum.head()

(25843, 6)


,id_student,code_module,code_presentation,weighted_score,total_weight_attempted,n_assessments
0,6516,AAA,2014J,63.50,100.0,5
1,8462,DDD,2013J,34.90,40.0,3
2,8462,DDD,2014J,43.00,50.0,4
3,11391,AAA,2013J,82.40,100.0,5
4,23629,BBB,2013B,16.69,25.0,4


In [15]:
student_df = studentInfo.merge(
    course_weighted_sum,
    on=["id_student", "code_module", "code_presentation"],
    how="left"
)

print("合并后形状:", student_df.shape)
student_df.head()

合并后形状: (32593, 15)


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,weighted_score,total_weight_attempted,n_assessments
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,82.4,100.0,5.0
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,65.4,100.0,5.0
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,NaN,NaN,NaN
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,76.3,100.0,5.0
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,55.0,100.0,5.0


In [16]:
course_df = student_df.merge(
    courses,
    on=["code_module", "code_presentation"],
    how="left"
)

print("合并后数据形状:", course_df.shape)
course_df.head()

合并后数据形状: (32593, 16)


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,weighted_score,total_weight_attempted,n_assessments,module_presentation_length
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,82.4,100.0,5.0,268
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,65.4,100.0,5.0,268
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,NaN,NaN,NaN,268
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,76.3,100.0,5.0,268
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,55.0,100.0,5.0,268


In [17]:
final_all = course_df.merge(
    studentRegistration,
    on=["id_student", "code_module", "code_presentation"],
    how="left"
)

print("合并后形状:", final_all.shape)
final_all.head()

合并后形状: (32593, 18)


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,weighted_score,total_weight_attempted,n_assessments,module_presentation_length,date_registration,date_unregistration
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,82.4,100.0,5.0,268,-159.0,NaN
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,65.4,100.0,5.0,268,-53.0,NaN
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,NaN,NaN,NaN,268,-92.0,12.0
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,76.3,100.0,5.0,268,-52.0,NaN
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,55.0,100.0,5.0,268,-176.0,NaN


In [18]:
df = final_all.merge(
    studentVle_final,
    on=["id_student", "code_module", "code_presentation"],
    how="left"
)

print("合并完成，df 形状:", df.shape)
df.head()


合并完成，df 形状: (32593, 21)


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,...,final_result,weighted_score,total_weight_attempted,n_assessments,module_presentation_length,date_registration,date_unregistration,total_clicks,first_day,last_day
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,...,Pass,82.4,100.0,5.0,268,-159.0,NaN,934.0,-5.0,253.0
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,...,Pass,65.4,100.0,5.0,268,-53.0,NaN,1435.0,-10.0,239.0
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,...,Withdrawn,NaN,NaN,NaN,268,-92.0,12.0,281.0,-10.0,12.0
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,...,Pass,76.3,100.0,5.0,268,-52.0,NaN,2158.0,-10.0,264.0
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,...,Pass,55.0,100.0,5.0,268,-176.0,NaN,1034.0,-10.0,247.0


In [19]:
# release the memory
datasets = None

### Save the dataset

In [20]:
df.to_csv("C:/Users/Jack/Desktop/GroupWork/Code&Data/Data/processed/processed.csv",
          index=False,
          encoding="utf-8-sig")

In [21]:
import pandas as pd

# 1. 读取数据（根据你的路径修改）
file_path = "C:/Users/Jack/Desktop/GroupWork/Code&Data/Data/processed/processed.csv"
df = pd.read_csv(file_path)

print("Base Information：")
print(df.info())

Base Information：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   code_module                 32593 non-null  object 
 1   code_presentation           32593 non-null  object 
 2   id_student                  32593 non-null  int64  
 3   gender                      32593 non-null  object 
 4   region                      32593 non-null  object 
 5   highest_education           32593 non-null  object 
 6   imd_band                    31482 non-null  object 
 7   age_band                    32593 non-null  object 
 8   num_of_prev_attempts        32593 non-null  int64  
 9   studied_credits             32593 non-null  int64  
 10  disability                  32593 non-null  object 
 11  final_result                32593 non-null  object 
 12  weighted_score              25843 non-null  float64
 13  total_weight_

In [22]:
# 2. 缺失值检测
missing_count = df.isnull().sum()
missing_rate = df.isnull().mean()

missing_df = pd.DataFrame({
    "missing_count": missing_count,
    "missing_rate": missing_rate
}).sort_values(by="missing_count", ascending=False)

print("\nMissing Value Statistics：")
print(missing_df)


Missing Value Statistics：
                            missing_count  missing_rate
date_unregistration                 22521      0.690977
n_assessments                        6750      0.207100
total_weight_attempted               6750      0.207100
weighted_score                       6750      0.207100
last_day                             3365      0.103243
first_day                            3365      0.103243
total_clicks                         3365      0.103243
imd_band                             1111      0.034087
final_result                            0      0.000000
date_registration                       0      0.000000
module_presentation_length              0      0.000000
code_module                             0      0.000000
code_presentation                       0      0.000000
studied_credits                         0      0.000000
num_of_prev_attempts                    0      0.000000
age_band                                0      0.000000
highest_education    

In [23]:
# 处理缺失 IMD_band（使用众数填充）
imd_missing_count = df['imd_band'].isnull().sum()
print(f"清洗前 imd_band 缺失值: {imd_missing_count}")

# 计算众数
mode_value = df['imd_band'].mode()[0]
print(f"用于填充的众数值: {mode_value}")

# 用众数填补缺失
df['imd_band'] = df['imd_band'].fillna(mode_value)

imd_missing_count_after = studentInfo['imd_band'].isnull().sum()
print(f"清洗后 imd_band 缺失值: {imd_missing_count_after}")

清洗前 imd_band 缺失值: 1111
用于填充的众数值: 20-30%
清洗后 imd_band 缺失值: 1111


In [24]:
# data unrefisteration will cause the issue of data leaking, delete them
df = df.drop(columns=["date_unregistration"])

In [25]:
# missing means that student did not view the resource, set them as zero
cols = ["last_day", "first_day", "total_clicks"]
df[cols] = df[cols].fillna(0)

# missing assessment means that student do not attend any assessment, set them as 0
cols = ["n_assessments", "total_weight_attempted", "weighted_score"]
df[cols] = df[cols].fillna(0)

In [26]:
missing_count = df.isnull().sum()
missing_count

code_module                   0
code_presentation             0
id_student                    0
gender                        0
region                        0
highest_education             0
imd_band                      0
age_band                      0
num_of_prev_attempts          0
studied_credits               0
disability                    0
final_result                  0
weighted_score                0
total_weight_attempted        0
n_assessments                 0
module_presentation_length    0
date_registration             0
total_clicks                  0
first_day                     0
last_day                      0
dtype: int64

In [27]:
# 3. 重复值检测
duplicate_count = df.duplicated().sum()

print("\nDuplicate Statistics:")
print(f"Number of duplicate rows: {duplicate_count}")

# 如果你想查看具体重复行（可选）
if duplicate_count > 0:
    print("\nSample duplicate rows:")
    print(df[df.duplicated()].head())



Duplicate Statistics:
Number of duplicate rows: 0


In [28]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   code_module                 32593 non-null  object 
 1   code_presentation           32593 non-null  object 
 2   id_student                  32593 non-null  int64  
 3   gender                      32593 non-null  object 
 4   region                      32593 non-null  object 
 5   highest_education           32593 non-null  object 
 6   imd_band                    32593 non-null  object 
 7   age_band                    32593 non-null  object 
 8   num_of_prev_attempts        32593 non-null  int64  
 9   studied_credits             32593 non-null  int64  
 10  disability                  32593 non-null  object 
 11  final_result                32593 non-null  object 
 12  weighted_score              32593 non-null  float64
 13  total_weight_attempted      325

### Process the categorical data (encoeding the data)

In [29]:
# 找出所有分类变量
categorical_cols = df.select_dtypes(include="object").columns

# 打印每个分类变量的类别
for col in categorical_cols:
    print(f"\n{col}: {df[col].nunique()}:")
    print(df[col].unique())



code_module: 7:
['AAA' 'BBB' 'CCC' 'DDD' 'EEE' 'FFF' 'GGG']

code_presentation: 4:
['2013J' '2014J' '2013B' '2014B']

gender: 2:
['M' 'F']

region: 13:
['East Anglian Region' 'Scotland' 'North Western Region'
 'South East Region' 'West Midlands Region' 'Wales' 'North Region'
 'South Region' 'Ireland' 'South West Region' 'East Midlands Region'
 'Yorkshire Region' 'London Region']

highest_education: 5:
['HE Qualification' 'A Level or Equivalent' 'Lower Than A Level'
 'Post Graduate Qualification' 'No Formal quals']

imd_band: 10:
['90-100%' '20-30%' '30-40%' '50-60%' '80-90%' '70-80%' '60-70%' '40-50%'
 '10-20' '0-10%']

age_band: 3:
['55<=' '35-55' '0-35']

disability: 2:
['N' 'Y']

final_result: 4:
['Pass' 'Withdrawn' 'Fail' 'Distinction']


| Variable Name     | Variable Type   | Number of Categories | Encoding Method  | Specific Processing                                                   |
| ----------------- | --------------- | -------------------- | ---------------- | --------------------------------------------------------------------- |
| code_module       | Nominal         | 7                    | One-Hot Encoding | One binary column per module (0/1)                                    |
| gender            | Binary          | 2                    | Binary Encoding  | M → 0, F → 1                                                          |
| region            | Nominal         | 13                   | One-Hot Encoding | One binary column per region (0/1)                                    |
| highest_education | Nominal         | 5                    | One-Hot Encoding | No artificial ordering imposed to avoid incorrect ordinal assumptions |
| imd_band          | Ordinal         | 10                   | Ordinal Encoding | Encoded from 0–10% → 0 up to 90–100% → 9 based on deprivation ranking |
| age_band          | Ordinal         | 3                    | Ordinal Encoding | 0–35 → 0, 35–55 → 1, ≥55 → 2                                          |
| disability        | Binary          | 2                    | Binary Encoding  | N → 0, Y → 1                                                          |
| final_result      | Target Variable | 4                    | Label Encoding   | Fail=0, Withdrawn=1, Pass=2, Distinction=3                            |


In [30]:
import pandas as pd

# ===== 1. 二值变量编码 =====
df['gender'] = df['gender'].map({'M': 0, 'F': 1})
df['disability'] = df['disability'].map({'N': 0, 'Y': 1})

# ===== 2. 有序变量编码 =====
imd_order = {
    '0-10%': 0, '10-20%': 1, '20-30%': 2, '30-40%': 3, '40-50%': 4,
    '50-60%': 5, '60-70%': 6, '70-80%': 7, '80-90%': 8, '90-100%': 9
}
df['imd_band'] = df['imd_band'].map(imd_order)

age_order = {
    '0-35': 0,
    '35-55': 1,
    '55<=': 2
}
df['age_band'] = df['age_band'].map(age_order)

# ===== 3. 目标变量 Label Encoding =====
final_map = {
    'Fail': 0,
    'Withdrawn': 1,
    'Pass': 2,
    'Distinction': 3
}
df['final_result'] = df['final_result'].map(final_map)

# ===== 4. 名义变量 One-Hot Encoding =====
nominal_cols = ['code_module', 'region', 'highest_education']
df = pd.get_dummies(df, columns=nominal_cols, drop_first=False)

# 查看结果
df.head()

,code_presentation,id_student,gender,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,weighted_score,...,region_South Region,region_South West Region,region_Wales,region_West Midlands Region,region_Yorkshire Region,highest_education_A Level or Equivalent,highest_education_HE Qualification,highest_education_Lower Than A Level,highest_education_No Formal quals,highest_education_Post Graduate Qualification
0,2013J,11391,0,9.0,2,0,240,0,2,82.4,...,False,False,False,False,False,False,True,False,False,False
1,2013J,28400,1,2.0,1,0,60,0,2,65.4,...,False,False,False,False,False,False,True,False,False,False
2,2013J,30268,1,3.0,1,0,60,1,1,0.0,...,False,False,False,False,False,True,False,False,False,False
3,2013J,31604,1,5.0,1,0,60,0,2,76.3,...,False,False,False,False,False,True,False,False,False,False
4,2013J,32885,1,5.0,0,0,60,0,2,55.0,...,False,False,False,True,False,False,False,True,False,False


### Normalization the data

In [31]:
df['total_day'] = df['last_day'] - df['first_day']
df.drop(columns=['last_day'], inplace=True)
df.drop(columns=['first_day'], inplace=True)

In [32]:
continuous_cols = [
    'num_of_prev_attempts',
    'studied_credits',
    'weighted_score',
    'total_weight_attempted',
    'n_assessments',
    'module_presentation_length',
    'date_registration',
    'total_clicks',
    'total_day',
    'imd_band',
    'age_band'
]

In [33]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[continuous_cols] = scaler.fit_transform(df[continuous_cols])

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 41 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   code_presentation                              32593 non-null  object 
 1   id_student                                     32593 non-null  int64  
 2   gender                                         32593 non-null  int64  
 3   imd_band                                       29077 non-null  float64
 4   age_band                                       32593 non-null  float64
 5   num_of_prev_attempts                           32593 non-null  float64
 6   studied_credits                                32593 non-null  float64
 7   disability                                     32593 non-null  int64  
 8   final_result                                   32593 non-null  int64  
 9   weighted_score                                 325

### The final CSV

In [35]:
df_J = df[df['code_presentation'].str.endswith('J')]
df_B = df[df['code_presentation'].str.endswith('B')]

# 保存为 CSV
df_J.to_csv('C:/Users/Jack/Desktop/GroupWork/Code&Data/Data/processed/J_group.csv', index=False)
df_B.to_csv('C:/Users/Jack/Desktop/GroupWork/Code&Data/Data/processed/B_group.csv', index=False)